## 03 - The True Power of Qurry

In previous section, we have learned how to use Qurry in basic way, it's kind of boring and not attractive for anyone, even you may ask why should I use this tool. In this section, we will learn how to use Qurry in advanced way, where the true power of Qurry lies.



In [1]:
from qurry import EntropyMeasure, BackendManager
from qiskit import (
    IBMQ, execute, transpile,
    QuantumRegister, ClassicalRegister, QuantumCircuit,
)
from pathlib import Path

experiment_executor_02 = EntropyMeasure()

## 3.1 - `BackendManager`, the advanced version `backendWrapper`

In previous chapter, we used `backendWrapper` to import our backend, and put provider input it to load IBM Backend from our IBM Account. 
Now, `BackendManager` will load accout automatically, and you can use `BackendManager` to load backend directly.

```python
BackendManager.save_account('your_token')
```
To save your account, if you have already saved, you can ignore this step.

```python
backend = BackendManager(
    instance='ibm-q/open/main'
)
```
or 
```python
backend = BackendManager(
    hub='ibm-q', group='open', project='main'
)
```


In [2]:
backend = BackendManager(
    hub='ibm-q', group='open', project='main'
)

| Provider by 'qiskit_ibm_provider'.


You can just type the name of backend without the prefix of `'ibmq_'` or `'ibm_'`.


In [3]:
backend('ibmq_belem'), backend('belem')

(<IBMBackend('ibmq_belem')>, <IBMBackend('ibmq_belem')>)

## 3.2 - Launching a multiJob

Consider a scenario where you have multiple circuit want to run on a backend, and you want to run them at the same time. In this case, you can use `multiOutput` to launch a multiJob.

For our example, we will show how to measure 100 circuits with 100 times randomized measure, 4096 shots, and mesure their entropy with multiple subsystem divisions at the same time.

### 1. Loading 100 circuits

(Topological Paramagnetic State is already the most complicated case in Qurry, so we will use it as our example, ususally I prefer some more complicated circuits like sshxx model with 20+ trotter steps, but it's not necessary for this example.)

```python

In [4]:
from qurry.case import topParamagnet
sample = topParamagnet(8, 'period')
print(sample.circuit)

     ┌───┐         
q_0: ┤ H ├─■─────■─
     ├───┤ │     │ 
q_1: ┤ H ├─■──■──┼─
     ├───┤    │  │ 
q_2: ┤ H ├─■──■──┼─
     ├───┤ │     │ 
q_3: ┤ H ├─■──■──┼─
     ├───┤    │  │ 
q_4: ┤ H ├─■──■──┼─
     ├───┤ │     │ 
q_5: ┤ H ├─■──■──┼─
     ├───┤    │  │ 
q_6: ┤ H ├─■──■──┼─
     ├───┤ │     │ 
q_7: ┤ H ├─■─────■─
     └───┘         


Loading circuits to `Qurry`

In [5]:
for i in range(100):
    experiment_executor_02.add(sample.circuit)


Check the name or serial number of the circuit in `Qurry` by `experiment_executor_02.waves.keys()`


In [6]:
print(experiment_executor_02.waves.keys())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])


### 2. Preparing configurations

For launch 100 experiment, it requires 100 configurations for each experiment, since they are similar, we can just make a list.

For randomized measure, each configuration is in this form of dictionary:

```python
{
    'waves': 'the circuit to measure',
    'tags': 'tags for this job',
    'shots': '...',
    'times': 100,
}
```

In [6]:
from typing import TypedDict, Hashable, Iterable, Union, Optional

class randomizedConfig(TypedDict):
    waves: Hashable
    """The name or serial number of circuit in Qurry."""
    tags: Union[Iterable[Hashable], Hashable]
    """You can metion tags to filter experiment."""
    times: Optional[int]
    """Default: 100 in :cls:`RandomizedMeasure`"""
    

In [7]:
configList: list[randomizedConfig] = [{
    'wave': i,
    'tags': ('topParamagnet', int(i/10)),
    'times': 100,
} for i in range(100)]

### 3. Launching multiJob

Attenetion, this example may make your computer go brrrr.

In [8]:
# please decoment this cell to run it.

# hashID = experiment_executor_02.multiOutput(
#     configList=configList,
#     # backend=backend('aer_gpu'),
#     backend=backend('aer'),
#     saveLocation=Path('./'),
#     summonerName='example.multiOutput',
#     shots=4096,
# )

# It takes 10m 41.2s to run this cell on my computer.

| MultiManager building...


| Write "example.multiOutput.qurry.001", at location "example.multiOutput.qurry.001": 

| Found the tarfile 'example.multiOutput.qurry.001.qurry.tar.xz' in '.', decompressing is available.
| No multi.config file found, decompressing all files in the tarfile 'example.multiOutput.qurry.001.qurry.tar.xz'.



| 100/100 100%|██████████| - Experiments build - 03:37 < 00:00


| Export multimanager...


...

| Export multimanager...


| 10/10 - exporting done:  - 00:04 < 00:00                      
| 0/0 - exporting quantity - 00:00 < ?


| Export multi.config.json for 08f2ee59-676a-4adb-a673-0a7a4be2e8dd


| 100/100 - Multimanger experiment write in 08f2ee59-676a-4adb-a673-0a7a4be2e8dd...done:  - 00:57 < 00:00                               


'example.multiOutput.qurry.001/multi.config.json' exported successfully.
| Compress multimanager of 'example.multiOutput.qurry.001'.../r'example.multiOutput.qurry.001/multi.config.json' exported successfully.
| Compress multimanager of 'example.multiOutput.qurry.001'...done


The `multiOutput` will return a ID of this multiJob, 

you can use it to check the status of this multiJob in `experiment_executor_02.multimanagers[(the ID returned)]`.

In [9]:
hashID # The hashID of this multiOutput

'08f2ee59-676a-4adb-a673-0a7a4be2e8dd'

In [10]:
experiment_executor_02.multimanagers

{'08f2ee59-676a-4adb-a673-0a7a4be2e8dd': <qurry.qurrium.multimanager.multimanager.MultiManager at 0x7f9214517410>}

### 4. The automatic export of multiJob

After the multiJob is finished, you may notice that there are some new files in your folder, they are the results of your multiJob.

Qurry will automatically export the results of multiJob to your folder, and you can use `multiRead` to load them. And we will show how to use `multiRead` in later section.

### 5. Make multiple analysis with multiple subsystem divisions

Attenetion, this example may make your computer go brrrr too.
And in randomized measure, this calculation already boosts by multiprocess and cython and makes it can be finished in one hour. In early version, such calculation can take all day, even few days to finish.

In [11]:
subsytems = [
    2, 4, 6, (2, 4), (4, 6), (2, 6), 
    (-2, 2), (-4, 2), (-2, 4), 3, (5, 7), (4, 7)]
# this is a list of subsystems that we want to measure

In [12]:
for d in subsytems:
    name = (
        f'N_A={"to".join([str(dd) for dd in d])}' if isinstance(d, tuple) 
        else f'N_A={d}')
    print("| Name:", name)
    
    # please decoment this cell to run it.
    
    # experiment_executor_02.multiAnalysis(
    #     summonerID=hashID,
    #     degree=d,
    #     analysisName=name,
    #     noSerialize=True,
    # )
    
# 38m 15.1s to run this cell on my computer.

| Name: N_A=2


| 100/100 - Analysis: Partition: (6, 8), Measure: (0, 8), 14 workers, 100 overlaps with mitigation. - 03:07 < 00:00


| "N_A=2" has been completed.
| Export multimanager...


| 10/10 - exporting done:  - 00:00 < 00:00                      
| 1/1 - exported quantity complete:  - 00:00 < 00:00


| Export multi.config.json for 08f2ee59-676a-4adb-a673-0a7a4be2e8dd
| Name: N_A=4


...

| Export multi.config.json for 08f2ee59-676a-4adb-a673-0a7a4be2e8dd
| Name: N_A=4to7


| 100/100 - Analysis: Partition: (4, 7), Measure: (0, 8), 14 workers, 100 overlaps with mitigation. - 03:11 < 00:00


| "N_A=4to7" has been completed.
| Export multimanager...


| 10/10 - exporting done:  - 00:00 < 00:00                      
| 12/12 - exported quantity complete:  - 00:00 < 00:00

| Export multi.config.json for 08f2ee59-676a-4adb-a673-0a7a4be2e8dd


## 3.3 Reading and Writing Data

In [6]:
hashID = experiment_executor_02.multiRead(
    saveLocation=Path('./'),
    summonerName='read.example.multiOutput.qurry.001',
)

| MultiRead running...
| Retrieve example.multiOutput.qurry.001...
| at: example.multiOutput.qurry.001
| Found the tarfile 'example.multiOutput.qurry.001.qurry.tar.xz' in '.', decompressing is available.
The following files '['example.multiOutput.qurry.001/N_A=2to6.quantity.json']' are fitting giving 'name' and 'additionName', choosing the 'example.multiOutput.qurry.001/N_A=2to6.quantity.json'.
The following files '['example.multiOutput.qurry.001/N_A=4to7.quantity.json']' are fitting giving 'name' and 'additionName', choosing the 'example.multiOutput.qurry.001/N_A=4to7.quantity.json'.


## 3.4 Export and Compress

In [7]:
experiment_executor_02.multiWrite(
    saveLocation=Path('./'),
    summonerID=hashID,
    compress=True,
)

| Export multimanager...


| 10/10 - exporting done:  - 00:04 < 00:00                      
| 12/12 - exported quantity complete:  - 00:00 < 00:00  


| Export multi.config.json for 08f2ee59-676a-4adb-a673-0a7a4be2e8dd


| 100/100 - Multimanger experiment write in 08f2ee59-676a-4adb-a673-0a7a4be2e8dd...done:  - 00:08 < 00:00                               


'example.multiOutput.qurry.001/multi.config.json' exported successfully.
| Compress multimanager of 'example.multiOutput.qurry.001'.../r'example.multiOutput.qurry.001/multi.config.json' exported successfully.
| Compress multimanager of 'example.multiOutput.qurry.001'...done


'08f2ee59-676a-4adb-a673-0a7a4be2e8dd'